# DQN

In [1]:
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.autograd as autograd
import math, random

In [2]:
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)
USE_CUDA = torch.cuda.is_available()

In [171]:
class QLearner(nn.Module):
    def __init__(self, env, num_frames, batch_size, gamma, replay_buffer):
        super(QLearner, self).__init__()

        self.batch_size = batch_size
        self.gamma = gamma
        self.num_frames = num_frames
        self.replay_buffer = replay_buffer
        self.env = env
        self.input_shape = self.env.observation_space.shape
        self.num_actions = self.env.action_space.n

        self.features = nn.Sequential(
            nn.Conv2d(self.input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, self.num_actions)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    def feature_size(self):
            return self.features(autograd.Variable(torch.zeros(1, *self.input_shape))).view(1, -1).size(1)
    
    def act(self, state, epsilon):
        action = []
        
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), requires_grad=True)
            ######## YOUR CODE HERE! ########
            # TODO: Given state, you should write code to get the Q value and chosen action
            # Complete the R.H.S. of the following 2 lines and uncomment them
            q_value = self.forward(state)
            action = torch.argmax(q_value)
            ######## YOUR CODE HERE! ########
        else:
            action = random.randrange(self.env.action_space.n)
        return action
        
def compute_td_loss(model, batch_size, gamma, replay_buffer):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)
    
    state = Variable(torch.FloatTensor(np.float32(state)), requires_grad=True)
    next_state = Variable(torch.FloatTensor(np.float32(next_state)), requires_grad=True)
    action = Variable(torch.LongTensor(action))
    reward = Variable(torch.FloatTensor(reward))
    done = Variable(torch.FloatTensor(done))

    ######## YOUR CODE HERE! ########
    # TODO: Implement the Temporal Difference Loss
    q_value = model.forward(state)
    next_q_value = model.forward(next_state)
    
    target = reward + gamma*torch.max(next_q_value)
    current = [q_value[ii,act] for ii, act in enumerate(action)]
    current = Variable(torch.FloatTensor(np.float32(current)), requires_grad=True)
    
    loss = torch.sqrt(torch.mean((current - target)**2))
    ######## YOUR CODE HERE! ########
    return loss


class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity) #Returns a new deque object initialized left-to-right

    def push(self, state, action, reward, next_state, done):
        state = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)

        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        ######## YOUR CODE HERE! ########
        # TODO: Randomly sampling data with specific batch size from the buffer
        # Hint: you may use the python library "random".
        
        batch = random.sample(self.buffer, batch_size)
        state  = []
        action = []
        reward = []
        next_state = []
        done = []
        for sample in batch:
            state.append(sample[0])
            action.append(sample[1])
            reward.append(sample[2])
            next_state.append(sample[3])
            done.append(sample[4])
        
        # If you are not familiar with the "deque" python library, please google it.
        ######## YOUR CODE HERE! ########
        return np.concatenate(state), action, reward, np.concatenate(next_state), done

    def __len__(self):
        return len(self.buffer)



# run DQN Pong

In [172]:
from Wrapper.layers import *
from Wrapper.wrappers import make_atari, wrap_deepmind, wrap_pytorch
import math, random
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F
USE_CUDA = torch.cuda.is_available()
# from dqn import QLearner, compute_td_loss, ReplayBuffer

In [173]:
env_id = "PongNoFrameskip-v4"
env = make_atari(env_id)
env = wrap_deepmind(env)
env = wrap_pytorch(env)

In [174]:
# plot observation of env

In [175]:
# num_frames = 1000000
# batch_size = 32
# gamma = 0.99
    
# replay_initial = 10000
# replay_buffer = ReplayBuffer(100000)
# model = QLearner(env, num_frames, batch_size, gamma, replay_buffer)
# optimizer = optim.Adam(model.parameters(), lr=0.00001)
# if USE_CUDA:
#     model = model.cuda()

# epsilon_start = 1.0
# epsilon_final = 0.01
# epsilon_decay = 30000
# epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

# losses = []
# all_rewards = []
# episode_reward = 0

# state = env.reset()

In [178]:
num_frames = 1000000
batch_size = 32
gamma = 0.99
    
replay_initial = 10000
replay_buffer = ReplayBuffer(10000)

model = QLearner(env, num_frames, batch_size, gamma, replay_buffer)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
if USE_CUDA:
    model = model.cuda()

epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 30000
epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

losses = []
all_rewards = []
episode_reward = 0

state = env.reset()

In [ ]:
for frame_idx in range(1, num_frames + 1):

    epsilon = epsilon_by_frame(frame_idx)
    action = model.act(state, epsilon)
    
    next_state, reward, done, _ = env.step(action)
    replay_buffer.push(state, action, reward, next_state, done)
    
    state = next_state
    episode_reward += reward
    if done:
        state = env.reset()
        all_rewards.append(episode_reward)
        episode_reward = 0

    if len(replay_buffer) > replay_initial:
        loss = compute_td_loss(model, batch_size, gamma, replay_buffer)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.data.cpu().numpy())

    if frame_idx % 10000 == 0 and len(replay_buffer) <= replay_initial:
        print('#Frame: %d, preparing replay buffer' % frame_idx)
        print(len(replay_buffer))
    if frame_idx % 10000 == 0 and len(replay_buffer) > replay_initial:
        print('#Frame: %d, Loss: %f' % (frame_idx, np.mean(losses)))
        print('Last-10 average reward: %f' % np.mean(all_rewards[-10:]))

#Frame: 10000, preparing replay buffer
10000
#Frame: 20000, preparing replay buffer
10000
#Frame: 30000, preparing replay buffer
10000
#Frame: 40000, preparing replay buffer
10000
#Frame: 50000, preparing replay buffer
10000
#Frame: 60000, preparing replay buffer
10000
#Frame: 70000, preparing replay buffer
10000
#Frame: 80000, preparing replay buffer
10000
#Frame: 90000, preparing replay buffer
10000
#Frame: 100000, preparing replay buffer
10000
#Frame: 110000, preparing replay buffer
10000
#Frame: 120000, preparing replay buffer
10000
#Frame: 130000, preparing replay buffer
10000
#Frame: 140000, preparing replay buffer
10000
#Frame: 150000, preparing replay buffer
10000
#Frame: 160000, preparing replay buffer
10000
#Frame: 170000, preparing replay buffer
10000
#Frame: 180000, preparing replay buffer
10000
#Frame: 190000, preparing replay buffer
10000
#Frame: 200000, preparing replay buffer
10000
#Frame: 210000, preparing replay buffer
10000
#Frame: 220000, preparing replay buffer
100